In [3]:
from public_data.models import Ocsge, Commune, CouvertureSol
from django.db import connection

def execute(query, parameters=None):
    with connection.cursor() as cursor:
        cursor.execute(query, parameters)
        return cursor.fetchall()

city = Commune.objects.get(name="Auch")
bbox = city.mpoly.extent
city, bbox


(<Commune: Auch (32013)>,
 (0.486067241998179, 43.6175987830157, 0.647056360983016, 43.7076959675878))

In [5]:
query = f"""
SELECT
    ST_NPoints(mpoly) AS geometry,
    ST_NPoints(ST_SimplifyPreserveTopology(mpoly, 0.000000001)),
    ST_NPoints(ST_SimplifyPreserveTopology(mpoly, 0.00000001)),
    ST_NPoints(ST_SimplifyPreserveTopology(mpoly, 0.0000001)),
    ST_NPoints(ST_SimplifyPreserveTopology(mpoly, 0.000001)),
    ST_NPoints(ST_SimplifyPreserveTopology(mpoly, 0.00001)),
    ST_NPoints(ST_SimplifyPreserveTopology(mpoly, 0.0001)),
    ST_NPoints(ST_SimplifyPreserveTopology(mpoly, 0.001)),
    ST_NPoints(ST_SimplifyPreserveTopology(mpoly, 0.01))
FROM {Ocsge._meta.db_table} as o INNER JOIN {CouvertureSol._meta.db_table} as t ON t.code_prefix = o.couverture
WHERE mpoly && ST_MakeEnvelope(%s, %s, %s, %s, 4326)
ORDER BY RANDOM() DESC
LIMIT 10
"""

execute(query, parameters=bbox)

[(33, 33, 33, 32, 31, 27, 16, 10, 10),
 (11, 11, 11, 6, 6, 6, 5, 5, 5),
 (28, 28, 24, 16, 15, 14, 5, 5, 5),
 (31, 31, 30, 29, 19, 7, 5, 5, 5),
 (42, 42, 39, 26, 26, 18, 6, 5, 5),
 (344, 342, 311, 202, 167, 132, 58, 35, 35),
 (15, 15, 14, 10, 8, 6, 5, 5, 5),
 (13, 13, 12, 8, 7, 6, 5, 5, 5),
 (1357, 1343, 1257, 851, 768, 548, 264, 113, 87),
 (17, 17, 17, 11, 9, 7, 5, 5, 5)]

In [11]:
params = bbox + bbox
params

(0.486067241998179,
 43.6175987830157,
 0.647056360983016,
 43.7076959675878,
 0.486067241998179,
 43.6175987830157,
 0.647056360983016,
 43.7076959675878)

In [40]:
query = f"""
SELECT *, nb_mpoly - nb_intersection
FROM
(SELECT
    ST_NPoints(mpoly) as nb_mpoly,
    ST_NPoints(ST_Intersection(mpoly, b.box)) as nb_intersection
FROM 
    {Ocsge._meta.db_table} as o 
    INNER JOIN (SELECT ST_MakeEnvelope(%s, %s, %s, %s, 4326) as box) as b ON ST_Intersects(o.mpoly, b.box)) as foo
ORDER BY  nb_mpoly - nb_intersection DESC
LIMIT 10
"""

execute(query, bbox)

[(53587, 3277, 50310),
 (53564, 3275, 50289),
 (42205, 237, 41968),
 (42202, 236, 41966),
 (50729, 14713, 36016),
 (50859, 14845, 36014),
 (46353, 18703, 27650),
 (46350, 18701, 27649),
 (43255, 18705, 24550),
 (43231, 18738, 24493)]

In [41]:
query = (
    'select o.couverture_label, o.couverture, o.usage_label, o.usage, o.surface, o.year, '
    'st_AsGeoJSON(ST_SimplifyPreserveTopology(ST_Intersection(mpoly, b.box), 0.0001), 6, 0) '
    'FROM public_data_ocsge o INNER JOIN (SELECT ST_MakeEnvelope(%s, %s, %s, '
    '%s, 4326) as box) as b ON ST_Intersects(o.mpoly, b.box)  where o.year = %s'
)
params = bbox + (2019,)

execute(query, params)

[('Zones bâties',
  'CS1.1.1.1',
  'Tertiaire',
  'US3',
  Decimal('736.6151'),
  2019,
  '{"type":"Polygon","coordinates":[[[0.59035,43.662802],[0.590441,43.663339],[0.590589,43.663316],[0.590452,43.662787],[0.59035,43.662802]]]}'),
 ('Zones bâties',
  'CS1.1.1.1',
  'Aérien',
  'US4.1.3',
  Decimal('343.6497'),
  2019,
  '{"type":"Polygon","coordinates":[[[0.603182,43.687558],[0.603029,43.687557],[0.60305,43.687781],[0.603232,43.687772],[0.603182,43.687558]]]}'),
 ('Zones bâties',
  'CS1.1.1.1',
  'Résidentiel',
  'US5',
  Decimal('490.6772'),
  2019,
  '{"type":"Polygon","coordinates":[[[0.584983,43.667485],[0.584844,43.667347],[0.584647,43.667479],[0.584785,43.66765],[0.584983,43.667485]]]}'),
 ('Zones bâties',
  'CS1.1.1.1',
  'Résidentiel',
  'US5',
  Decimal('282.0988'),
  2019,
  '{"type":"Polygon","coordinates":[[[0.585037,43.667865],[0.585013,43.667967],[0.58533,43.668001],[0.585353,43.667915],[0.585037,43.667865]]]}'),
 ('Zones bâties',
  'CS1.1.1.1',
  'Résidentiel',
  'US5

In [31]:
execute("SELECT ST_Area(ST_Transform(mpoly, 2154)) / 1000000 FROM public_data_ocsge LIMIT 10")

[(0.02606713800038099,),
 (0.0007366150999471778,),
 (1.4199999943445434e-07,),
 (0.026928143149434352,),
 (0.00034364974996407843,),
 (0.0005727533712357102,),
 (0.00020100978708928645,),
 (0.0004906772498956491,),
 (0.005079992450239695,),
 (0.008801632999959065,)]

In [6]:
execute("""
SELECT 
    (ST_Dump(
      ST_MakeGrid(
        ST_Transform(
          ST_SetSRID(ST_Extent(mpoly), 4326),
          2154
        ),
        1000, -- Taille de la cellule en mètres
        true -- Retourner uniquement les cellules entièrement couvertes
      )
    )).geom AS cell_geom
FROM public_data_ocsge
LIMIT 10
""")

ProgrammingError: function st_makegrid(geometry, integer, boolean) does not exist
LINE 4:       ST_MakeGrid(
              ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.


In [36]:
execute("""
SELECT ST_SnapToGrid(mpoly, 0.01) AS mpoly
FROM public_data_ocsge, 
generate_series( 
  ST_XMin(ST_Envelope(mpoly)), 
  ST_XMax(ST_Envelope(mpoly)), 
  0.01
) xGrid(x), 
generate_series( 
  ST_YMin(ST_Envelope(mpoly)), 
  ST_YMax(ST_Envelope(mpoly)), 
  0.01
) yGrid(y)
WHERE ST_Intersects(mpoly, ST_MakeEnvelope(x, y, x + 0.01, y + 0.01, 4326))
""")

ProgrammingError: function generate_series(double precision, double precision, numeric) does not exist
LINE 4: generate_series( 
        ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.


In [46]:
execute("""
SELECT squares.geom
FROM
ST_SquareGrid(
    1000,
    ST_Transform(ST_MakeEnvelope(%s, %s, %s, %s, 4326), 2154)
) AS squares
""", bbox)

[('...',)]

In [54]:
from django.core.serializers import serialize
import json

query = (
    "SELECT st_AsGeoJSON(squares.geom, 6, 0) as mpoly "
    "FROM ST_SquareGrid(%s, ST_Transform(ST_MakeEnvelope(%s, %s, %s, %s, 4326), 2154)) AS squares"
)
params = (1000,) + bbox

with connection.cursor() as cursor:
    cursor.execute(query, params)
    geojson = {
        "type": "FeatureCollection",
        "crs": {"type": "name", "properties": {"name": "EPSG:4326"}},
        "features": [
            {
                "type": "Feature",
                "geometry": json.loads(row[0]),
            } for row in cursor.fetchall()
        ],
    }

geojson

{'type': 'FeatureCollection',
 'crs': {'type': 'name', 'properties': {'name': 'EPSG:4326'}},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[497000, 6282000],
      [497000, 6283000],
      [498000, 6283000],
      [498000, 6282000],
      [497000, 6282000]]]}},
  {'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[497000, 6283000],
      [497000, 6284000],
      [498000, 6284000],
      [498000, 6283000],
      [497000, 6283000]]]}},
  {'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[497000, 6284000],
      [497000, 6285000],
      [498000, 6285000],
      [498000, 6284000],
      [497000, 6284000]]]}},
  {'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[497000, 6285000],
      [497000, 6286000],
      [498000, 6286000],
      [498000, 6285000],
      [497000, 6285000]]]}},
  {'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[497000, 6286

In [ ]:
try:
            envelope = json.dumps(
                {
                    "type": "FeatureCollection",
                    "crs": {"type": "name", "properties": {"name": "EPSG:4326"}},
                    "features": "-features-",
                }
            )
            features = []
            for row in self.get_data(request):
                geojson = row.pop("geojson")
                feature = json.dumps(
                    {
                        "type": "Feature",
                        "properties": self.clean_properties(row),
                        "geometry": "-geometry-",
                    },
                    default=decimal2float,
                )
                feature = feature.replace('"-geometry-"', geojson)
                features.append(feature)
            features = f" [{', '.join(features)}]"
            envelope = envelope.replace('"-features-"', features)
        except ValueError as exc:
            envelope = json.dumps(
                {
                    "error": {
                        "type": "ValueError",
                        "message": str(exc),
                    }
                }
            )
        return HttpResponse(envelope, content_type="application/json")

In [17]:
execute("""
    SELECT o.id, ST_AsText(mic.center), ST_AsEWKT(mic.center)
FROM 
    public_data_ocsge as o,
    LATERAL ST_MaximumInscribedCircle(o.mpoly) AS mic
LIMIT 10
""")

[(710279,
  'POINT(0.038190497641317 43.70419546437574)',
  'SRID=4326;POINT(0.038190497641317 43.70419546437574)'),
 (924637,
  'POINT(0.590397113987606 43.66287868370952)',
  'SRID=4326;POINT(0.590397113987606 43.66287868370952)'),
 (665382,
  'POINT(0.301641796169934 43.74128899696824)',
  'SRID=4326;POINT(0.301641796169934 43.74128899696824)'),
 (710291,
  'POINT(-0.002081228926743 43.706033343015775)',
  'SRID=4326;POINT(-0.002081228926743 43.706033343015775)'),
 (924664,
  'POINT(0.603126430707722 43.687692349139056)',
  'SRID=4326;POINT(0.603126430707722 43.687692349139056)'),
 (100845,
  'POINT(-0.767636932332346 44.60742447152698)',
  'SRID=4326;POINT(-0.767636932332346 44.60742447152698)'),
 (150459,
  'POINT(-1.230902661628691 44.549126528839174)',
  'SRID=4326;POINT(-1.230902661628691 44.549126528839174)'),
 (924699,
  'POINT(0.584808585925596 43.66749583745671)',
  'SRID=4326;POINT(0.584808585925596 43.66749583745671)'),
 (710305,
  'POINT(0.064796606232583 43.710864306344

In [8]:
execute("""SELECT radius, ST_AsText(center) AS center, ST_AsText(nearest) AS nearest
    FROM ST_MaximumInscribedCircle(
        'POLYGON ((40 180, 110 160, 180 180, 180 120, 140 90, 160 40, 80 10, 70 40, 20 50, 40 180),
        (60 140, 50 90, 90 140, 60 140))');""")

[(45.165845650018, 'POINT(96.953125 76.328125)', 'POINT(140 90)')]

In [18]:
execute("""
SELECT o.id, (ST_MaximumInscribedCircle(o.mpoly)).center
FROM 
    public_data_ocsge as o
LIMIT 10
""")

[(710279, '0101000020E61000003C5CC674B48DA33F20BEB41323DA4540'),
 (924637, '0101000020E61000007958077D88E4E23FBCDF6D35D9D44540'),
 (665382, '0101000020E61000002F056A64194ED33F7E6ECF8EE2DE4540'),
 (710291, '0101000020E6100000468FA045A70C61BFB411F34C5FDA4540'),
 (924664, '0101000020E6100000A4C5E7CCCF4CE33F7EA18A4D06D84540'),
 (100845, '0101000020E6100000D036F2537B90E8BFCEFFC715C04D4640'),
 (150459, '0101000020E6100000C51244FDC6B1F3BF795D31C749464640'),
 (924699, '0101000020E6100000DA0DDF7EC0B6E23FDE0BEC8070D54540'),
 (710305, '0101000020E6100000D325A9A88296B03F26D2019AFDDA4540'),
 (710306, '0101000020E6100000CD190F70E241B03FB30BA46EFDDA4540')]